### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [507]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [559]:
#%%pycodestyle

class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (l_c ** 2 / l_s + r_c ** 2 / r_s).sum(axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return - np.sum((l_c / l_c + r_c / r_s) * log(l_c / l_c + r_c / r_s))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c / (l_s + r_s), axis=1)
                    + np.max(r_c / (l_s + r_s), axis=1))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature)]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature)]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x, y = self.__sort_samples(x, y)
        cut_size = np.int(self.min_samples_split / 2 - 1)

        if cut_size == 0:
            split_y = y
        else:
            split_y = y[cut_size:-cut_size]
        border = np.where(split_y[:-1] != split_y[1:])[0] + (cut_size + 1)

        if len(border) == 0:
            return np.inf, None

        count = border - np.append(np.array([cut_size]), border[:-1])
        class_matrix = np.zeros((border.shape[0], self.num_class))
        class_matrix[np.arange(border.shape[0]), y[border - 1]] = 1
        class_increments = class_matrix * count.reshape(-1, 1)

        l_c = np.cumsum(class_increments, axis=0)
        r_c = np.bincount(y, minlength=self.num_class) - l_c
        l_s = border.reshape(l_c.shape[0], 1)
        r_s = y.shape[0] - l_s

        gs = self.G_function(l_c, l_s, r_c, r_s)
        index = np.argmin(gs)

        id_ = l_s[index][0]
        return gs[index], (x[id_ - 1] + x[id_]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        if (self.max_depth is not None and self.max_depth <= depth) or\
            (y.shape[0] < self.min_samples_split) or \
            (self.sufficient_share <= np.bincount(y).max() / y.shape[0]):

            class_ = np.bincount(y, minlength=self.num_class)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, class_.argmax(),
                                  class_.astype(float) / y.size)
            return
        
        threshold = np.array([self.__find_threshold(x[:, f], y) for f in self.get_feature_ids(x.shape[1])])
        best_id = threshold[:, 0].argmin()
        best_threshold = threshold[best_id, 1]
        
        l_x, r_x, l_y, r_y = self.__div_samples(x, y, self.get_feature_ids(x.shape[1])[best_id], best_threshold)
        
 
        if l_x.shape[0] == 0 or r_x.shape[0] == 0:
            buf = np.bincount(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, buf.argmax(),
                                  buf.astype(float) / y.size)
            return

        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, best_id, best_threshold)          
        self.__fit_node(l_x, l_y, node_id * 2 + 1, depth + 1)
        self.__fit_node(r_x, r_y, node_id * 2 + 2, depth + 1)
        
        if self.G_function == self.__gini:
                func = 1 - np.sum((np.bincount(y) / y.size) ** 2)

        if self.G_function == self.__entropy:
                func = - np.sum((np.bincount(y) / y.size) * (np.log(np.bincount(y) / y.size)))
 
        if self.G_function == self.__misclass:
                func = 1 - np.max(np.bincount(y) / y.size)
        
        self.feature_importances_[best_id] += (func - threshold[:, 0].min())
        

    
    def fit(self, x, y):
        self.feature_importances_ = np.zeros(x.shape[1])
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [588]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [589]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [590]:
%time clf.fit(X_train, y_train)

CPU times: user 1.97 ms, sys: 811 µs, total: 2.78 ms
Wall time: 2.6 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [591]:
%time my_clf.fit(X_train, y_train)

CPU times: user 18.3 ms, sys: 1.72 ms, total: 20 ms
Wall time: 18.6 ms


## Проверка качества работы на wine

In [592]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.7634920634920634

In [593]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8278943278943279

## Подготовка данных Speed Dating Data 

In [566]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', sep=',', encoding="ISO-8859-1")
df = df.drop(['id', 'condtn', 'idg', 'round', 'condtn', 'position', 'positin1',
              'partner', 'race_o', 'age_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int',
              'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'attr_o', 'sinc_o', 'intel_o', 'dec_o', 'fun_o',
              'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age', 'mn_sat', 'undergra', 'tuition'], axis=1)


In [567]:
clf = DecisionTreeClassifier()
my_clf = MyDecisionTreeClassifier()

In [568]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Проверка скорости работы на Speed Dating Data 

In [569]:
%time clf.fit(X_train, y_train)

CPU times: user 79.2 ms, sys: 1.68 ms, total: 80.8 ms
Wall time: 79.8 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [570]:
%time my_clf.fit(X_train, y_train)

CPU times: user 853 ms, sys: 9.99 ms, total: 863 ms
Wall time: 865 ms


## Проверка качества работы на Speed Dating Data

In [571]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5428571428571428

In [572]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.4909271912756233

## Задание 3

## Задание 4

In [573]:
df_pair.columns[np.argsort(clf.feature_importances_)][:-10:-1]

Index(['match', 'race_f', 'amb1_1_f', 'field_cd_f', 'amb1_1', 'goal_f',
       'career_c', 'exphappy', 'sinc1_1_f'],
      dtype='object')

In [574]:
df_pair.columns[np.argsort(my_clf.feature_importances_)][:-10:-1]

Index(['goal', 'amb1_1_f', 'fun1_1', 'shar2_1', 'intel1_1', 'samerace',
       'shar1_1', 'career_c', 'date_f'],
      dtype='object')

## Задание 5

In [575]:
params = {'n_estimators': [10, 20, 30, 50 , 70, 100, 150],
          'max_depth': [5, 10, 20],
          'min_samples_leaf': [1, 3, 5, 7, 10]}

model = RandomForestClassifier()

clf = RandomizedSearchCV(model, param_distributions=params)
clf.fit(X, y)
clf.best_params_

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'n_estimators': 100, 'min_samples_leaf': 5, 'max_depth': 10}